In [70]:
import os
from dotenv import load_dotenv
load_dotenv()
from grok import Grok
import pickle

In [71]:
grok_api_key, grok_url, grok_model = os.getenv("GROK_API_KEY"), os.getenv("GROK_URL"), 'grok-2-latest'
print(grok_api_key, grok_url)
grok = Grok(grok_api_key, grok_url, grok_model)

xai-fq2OPXVKJrQCMyitw8z4fUiuVE6OO0KBjy1AzT5L56DiZsG88SqNJgvISwXC7psVI6wmT5vkAkFWadXF https://api.x.ai/v1/chat/completions


In [140]:
def generate_script_grok(client: Grok, trend_data: dict, news_report: str) -> str:

    """Generate a video script using Grok based on trend data"""
    
    prompt = f"""Create a short, video script about this trending topic:
    Topic: {trend_data['keyword']}
    Related Keywords: {', '.join(trend_data['keywords'])}
    News Report: {news_report}
    
    The script must be:
    1. Informative and accurate
    2. Focused on the most important and sensational aspects of the story
    3. Conversational and concise.
    4. Written with charged language and emotional appeals.
    5. Quick to the point, no fluff.
    6. Between 60 and 80 words in length.

    The script must NOT:
    1. Contain an outro of any kind, or mention subscribing to the channel
    2. Use acronyms or abbreviations without explanation
    3. Contain any headers, formatting, directions for the video, or other instructions.

    End the script with a uniqueand engaging question.
    """
    return client.generate_text(prompt)

In [73]:
def generate_sora_prompts_grok(client: Grok, script: str, num_prompts: int = 1) -> list[str]:
    """Generate Sora-compatible video prompts based on the script"""

    soraScripts = []
    prompt = f"""Break down this script into a series of four video scene descriptions suitable for AI video generation:
    
    Script: {script}
    
    There should be four scenes.

    For each scene:
    
    1. Describe the visual elements in detail
    2. Include camera movements and transitions
    3. Be concise -- under 25 words.
    4. Use loose, conversational language in your description.
    5. Format the scene description as follows:
        [Scene Title] <= 5 words
        [Scene Description] <= 25 words
        [Camera Movements/Transitions] <= 25 words
        newline

    Do NOT:
    1. Generate a scene for an outro
    2. Use acronyms or abbreviations without explanation
    """
    
    for i in range(num_prompts):
        soraScript = client.generate_text(prompt)
        soraScripts.append(soraScript)
    return soraScripts

In [74]:
def generate_sora_prompts_grokV2(client: Grok, script: str, num_prompts: int = 1) -> list[str]:
    """Generate Sora-compatible video prompts based on the script"""

    soraScripts = []
    prompt = f"""Break down this script into a series of four video scene descriptions suitable for AI video generation:
    
    Script: {script}
    
    There should be four scenes.

    For each scene:
    
    1. Describe the visual elements in detail
    2. Include camera movements and transitions
    3. Be concise -- under 25 words.
    4. Use loose, conversational language in your description.
    5. Scene descriptions should tell a story. Ensure that it can be told in ten seconds or less.
    6. Format the scene description as follows:
        [Scene Title] <= 5 words
        [Scene Description] <= 25 words
        [Camera Movements/Transitions] <= 25 words
        newline

    Do NOT:
    1. Generate a scene for an outro
    2. Use acronyms or abbreviations without explanation
    """
    
    for i in range(num_prompts):
        soraScript = client.generate_text(prompt)
        soraScripts.append(soraScript)
    return soraScripts

In [75]:
def generate_grok_image_prompts(client: Grok, script: str, num_prompts: int = 10) -> list[str]:
    """Generate Grok image prompts based on the script"""

    prompt = f"""Generate a series of (~{int}) image descriptions that capture the most sensational themes of the following script.

    For each image:
    1. Be assertive and highly creative in your description. 
    2. Be clear and concise. 
    3. Use familiar elements of popular culture where appropriate e.g. brands, celebrities, landmarks, products, etc.

    Each image prompt should have the following format:
    Generate an image of [description]

    Separate each image prompt with a newline.
    
    Script: {script}
    """
    return client.generate_text(prompt)

In [76]:
def generate_grok_narration(client: Grok, script: str) -> str:
    """Generate a narration using Grok based on the script"""

    prompt = f"""Extract only the words to be spoken from this video script:
    Script: {script}.

    The result should NOT include any mention of a narrator or scene descriptions.
    """
    return grok.generate_text(prompt)

In [77]:
def generate_hashtags(client: Grok, script: str) -> list[str]:
    """Generate hashtags using Grok based on the script"""

    prompt = f"""Generate the best series of hashtags to make this video go viral on instagram reels, tik tok, and youtube shorts.
    Select the best 10 hashtags and brief reasoning for each.

    The result should be in the following format:
    for each hashtag:
        [Hashtag] [Reasoning]

    Include a one-line space separated list of all hashtags at the end.

    Script: {script}"""
    return client.generate_text(prompt)


In [78]:
def generate_grok_opinion_analysis(client: Grok, news_report: str) -> str:
    """Generate an opinion analysis using Grok based on the news report"""

    prompt = f"""Generate an opinion analysis using Grok based on the news report:
    News Report: {news_report}

    The opinion analysis should:
    1. Be highly engaging and bold.
    2. Provide evidence based reasoning for claims made. 
    3. Be less than 500 words in length.
    4. Be written in a professional tone.

    The opinion analysis should NOT:
    1. Cite sources.

    Format:
    [Title Highlighting the Central Claim] <= 10 words
    [Opinion Analysis] <= 500 words
    """

    return client.generate_text(prompt)

In [79]:
grok_api_key, grok_url, grok_model = os.getenv("GROK_API_KEY"), os.getenv("GROK_URL"), 'grok-2-latest'
grok = Grok(grok_api_key, grok_url, grok_model)

In [80]:
# List directories in assets and get most recent
asset_dirs = [d for d in os.listdir('assets') if os.path.isdir(os.path.join('assets', d))]
RUN_DIR = os.path.join('assets', max(asset_dirs))
MAIN = [os.path.join(RUN_DIR, d) for d in os.listdir(RUN_DIR) if os.path.isdir(os.path.join(RUN_DIR, d))][0] # only one directory in the RUN_DIR
NEWS_REPORT = os.path.join(MAIN, 'news_report.txt')
SORAPROMPTS = os.path.join(MAIN, 'soraprompts.txt')
SCRIPT = os.path.join(MAIN, 'script.txt')
keyword = MAIN.split('\\')[-1].replace('_', ' ')

In [81]:
finalTrendList = pickle.load(open('./tmp/finalTrendList.pkl', 'rb'))

In [82]:
trend = [trend for trend in finalTrendList if trend['keyword'] == keyword][0]

In [83]:
with open(NEWS_REPORT) as f:
    newsReport = f.read()
print(newsReport)

**Binance’s Future in the Spotlight: Investment Talks with Trump Family Amid Ongoing Controversies**

*March 13, 2025 | By Finance Today Reporters*

In a significant week for cryptocurrency giant Binance, the company finds itself at a crossroads as multiple developments unfold that could define its future in the U.S. market. This comes amid reports that representatives of the Trump family have had discussions to secure a financial stake in Binance.US, further entangling the exchange in political and ethical conversations.

### Investment Talks with the Trump Family

According to a report from the *Wall Street Journal*, negotiations between Binance executives and the Trump family signify the exchange's attempts to reclaim its footing in the U.S. This has raised eyebrows, as Binance’s founder Changpeng Zhao (CZ) has been on the lookout for a presidential pardon following his 2023 guilty plea concerning anti-money laundering violations. Zhao, who served four months in prison, denied the r

In [84]:
GROKPATH = MAIN + '/grok'
os.mkdir(GROKPATH)

In [85]:
soraPromptPath = GROKPATH + '/grok-soraprompts.txt'
soraPromptsv2Path = GROKPATH + 'grok-soraprompts-v2.txt'
scriptPath = GROKPATH + '/grok-script.txt'
grokImagePromptsPath = GROKPATH + '/grok-imageprompts.txt'
grokNarrationPath = GROKPATH + '/grok-narration.txt'
grokHashtagsPath = GROKPATH + '/grok-hashtags.txt'
opinionAnalysisPath = GROKPATH + '/grok-opinionanalysis.txt'

In [141]:
narrationWC = 0
while narrationWC < 60 or narrationWC > 80:
    script = generate_script_grok(grok, trend, newsReport)
    narration = generate_grok_narration(grok, script)
    narrationWC = len(narration.split(' '))
    print('narrationWC: ', narrationWC)
    print(narration)
    print("=========")
with open(scriptPath, 'w') as f:
    f.write(script)
with open(grokNarrationPath, 'w') as f:
    f.write(narration)

narrationWC:  64
Binance is making headlines again, this time with shocking news of investment talks with the Trump family! Amidst ongoing controversies and a guilty plea from founder Changpeng Zhao, or CZ, for anti-money laundering violations, the stakes are higher than ever. The potential partnership could reignite Binance's presence in the U.S., but at what cost? Will this controversial move restore trust or deepen the crisis?
Binance is making headlines again, this time with shocking news of investment talks with the Trump family! Amidst ongoing controversies and a guilty plea from founder Changpeng Zhao, or CZ, for anti-money laundering violations, the stakes are higher than ever. The potential partnership could reignite Binance's presence in the U.S., but at what cost? Will this controversial move restore trust or deepen the crisis?


In [142]:
images = generate_grok_image_prompts(grok, script)
with open(grokImagePromptsPath, 'w') as f:
    f.write(images)
print(images)

Generate an image of a dramatic headline on a giant digital billboard in Times Square, reading "Binance in Talks with Trump Family!" with the Binance logo and Trump's silhouette against a backdrop of flashing lights and bustling crowds.

Generate an image of Changpeng Zhao, or CZ, in a courtroom, pleading guilty to anti-money laundering charges, with dramatic shadows and a gavel in mid-strike, symbolizing the gravity of the situation.

Generate an image of the Binance logo intertwined with the Trump family crest, set against the backdrop of the U.S. Capitol, symbolizing a potential controversial partnership that could reignite Binance's U.S. presence.

Generate an image of a seesaw balancing the words "Trust" and "Crisis" on either end, with the Binance and Trump logos on opposite sides, set against a stormy sky, illustrating the high stakes and uncertain outcomes of the potential partnership.


In [143]:
soraPrompts = generate_sora_prompts_grok(grok, script)
with open(soraPromptPath, 'w') as f:
    f.write(soraPrompts[0])
print(soraPrompts[0])

[Binance in the News]
Visual: Newspaper headlines about Binance and Trump family investment talks.
Camera: Zoom in on headlines, pan to next scene.
newline

[Controversy and Guilty Plea]
Visual: Changpeng Zhao in court, looking serious, with anti-money laundering text.
Camera: Slow zoom on Zhao, cut to next scene.
newline

[High Stakes in the U.S.]
Visual: Binance logo with U.S. flag, question marks around.
Camera: Rotate around logo, zoom out to next scene.
newline

[Trust or Crisis Deepens?]
Visual: Scales balancing trust and crisis, Binance and Trump logos.
Camera: Tilt up from scales, fade out.
newline


In [144]:
soraPrompts2 = generate_sora_prompts_grokV2(grok, script)
with open(soraPromptsv2Path, 'w') as f:
    f.write(soraPrompts2[0])
print(soraPrompts2[0])

[Binance in the News]
Headlines flash across screens, focusing on Binance and Trump family investment talks.
Camera zooms in on headlines, quick cuts between news clips.

[Binance Founder's Guilty Plea]
Changpeng Zhao in a courtroom, looking serious, as a judge announces his guilty plea for anti-money laundering violations.
Camera pans from Zhao to judge, then fades out.

[Potential U.S. Partnership]
Binance logo next to Trump family silhouette, symbolizing potential partnership, with U.S. flag in background.
Camera slowly zooms in on the logo and silhouette, then cuts.

[Trust or Crisis Deepens?]
Split screen showing Binance's logo and a question mark, representing the uncertainty of the partnership's impact.
Camera starts wide, zooms in on question mark, then dissolves to next scene.


In [146]:
hashtags = generate_hashtags(grok, script)
with open(grokHashtagsPath, 'w') as f:
    f.write(hashtags)
print(hashtags)


1. **#BinanceTrump**
   Reasoning: Directly references the key elements of the video - Binance and the Trump family, making it highly relevant and searchable for those interested in this specific news.

2. **#CZGuiltyPlea**
   Reasoning: Highlights the significant legal development involving Binance's founder, which is a major point of interest and discussion among viewers.

3. **#CryptoControversy**
   Reasoning: Broadens the appeal to those interested in cryptocurrency controversies, which is a hot topic in the financial and tech communities.

4. **#BinanceUS**
   Reasoning: Targets viewers specifically interested in Binance's activities and potential moves in the U.S. market.

5. **#TrumpFamilyInvestments**
   Reasoning: Appeals to audiences following the Trump family's business dealings, which often generate significant public interest and discussion.

6. **#AntiMoneyLaundering**
   Reasoning: Focuses on the specific legal issue at hand, attracting viewers interested in financial r

In [147]:
opinionAnalysis = generate_grok_opinion_analysis(grok, newsReport)

In [148]:
with open(opinionAnalysisPath, 'w') as f:
    f.write(opinionAnalysis)
print(opinionAnalysis)

**Binance's High-Stakes Gamble with Trump Ties**

Binance's audacious move to court the Trump family's investment through World Liberty Financial is a bold, yet risky strategy to reclaim its foothold in the U.S. market. This partnership, if realized, could provide the financial lifeline Binance desperately needs after its significant setbacks due to regulatory pressures and anti-money laundering sanctions. However, the potential benefits come with substantial risks, as the involvement of controversial political figures could further erode the trust Binance is struggling to rebuild.

The reported discussions with the Trump family, coupled with the recent $2 billion investment from MGX, demonstrate Binance's determination to solidify its position in the digital finance landscape. The surge in BNB's price following the Trump family news suggests investor optimism, but the broader market remains volatile amidst macroeconomic uncertainties. Binance's ability to navigate these challenges whi

In [149]:
len(grokNarration.split(' '))

64